# Selection Bias - Recidivism Data Measuring Race Equality

The code bellow is an implementation of measuring equality, meaning it measures the parity of output from a neural network. This implementation measuring equality assumes that there is a single output from a network, where outputs bellow .25 indicates “Low,” .25-.74” indicates “Medium” and above .75 indicates “High”. The target values from the training dataset are either 0 indicating “Low,” .5 indicating “Medium” or 1 indicating “High.” The code for measuring equality can be seen bellow in Figure 5. The “getBias” function in this code takes in “falseCount,” the count of false results in a test dataset and “testSet,” which is the test dataset that is used for its length.

The measuring equality technique was implemented on a single network. The network was trained with 4000 rows of data that only included cases of Caucasian and African-American, 1500 epochs of training, and a tolerance of .25. The percentage correctness of this network was 65%.

The results of this experienment are at the end of this file where the getBias function is run. They were generated from two test datasets that used the remaining 2149 rows of COMPAS data that the network was not trained on. The two datasets are identical except that they were both altered so that value was all Caucasian in one and all African American in the other. The motivation for doing this was to test how the network is influenced by race.

In [1]:
from conx import Network
import random

In [2]:
def getData(file):
    #open file
    tf = open(file, "r")
    content = tf.readlines()
    row_count = sum(1 for row in content)
    print(row_count)
    #Parse file
    input = []
    content = [x.strip() for x in content[1:]]
    for y in content:
        row = [col for col in y.split(",")]
        #print(row)
        input.append(row)
    tf.close()
    return input

cols = getData("compas-scores-two-years.csv")

7215


In [3]:
for c in cols:
    if len(c) == 54:
        c.remove(c[23])
        
print(len(cols[4062]))

53


In [4]:
def parseInputs(inputs):
    new = []
    row = []
    countWhat = 0
    for r in inputs:
        if r[9] == "Caucasian" or r[9] == "African-American":
            if r[5] == "Male":
                row.append(1)
            elif r[5] == "Female":
                row.append(0)
            else:
                print("error parsing col 5, Error occurs on row " + str(countWhat))
            #Age: Greater than 45, 25 - 45, and Less than 25
            if r[8] == "Greater than 45":
                row.append(1)
                row.append(0)
                row.append(0)
            elif r[8] == "25 - 45":
                row.append(0)
                row.append(1)
                row.append(0)
            elif r[8] == "Less than 25":
                row.append(0)
                row.append(0)
                row.append(1)
            else:
                print("error parsing col 8, Error occurs on row " + str(countWhat))
            #race = Caucasian, African-American
            if r[9] == "Caucasian":
                row.append(0)
            elif r[9] == "African-American":
                row.append(1)
            else:
                print("error parsing col 9, Error occurs on row " + str(countWhat))
            #Risk of violence Score: Low, Medium, and High
            if r[44] == "Low":
                row.append(1)
                row.append(0)
                row.append(0)
            elif r[44] == "Medium":
                row.append(0)
                row.append(1)
                row.append(0)
            elif r[44] == "High":
                row.append(0)
                row.append(0)
                row.append(1)
            else:
                print("error parsing col 44, Error occurs on row " + str(countWhat))  
            #Risk recidivism Score: Low, Medium, and High
            if r[40] == "Low":
                row.append(0)
            elif r[40] == "Medium":
                row.append(.5)
            elif r[40] == "High":
                row.append(1)
            else:
                print("error parsing col 40, Error occurs on row " + str(countWhat))
            #parse error in row 4754, disposed of here
            if r[40] == "":
                r = []
            if row != [] and len(row) == 9:
                new.append(row)
            #reset
            row = []
        countWhat += 1
    return new

tempinputs = parseInputs(cols)  
tempinputs[0]

error parsing col 40, Error occurs on row 4754


[1, 0, 1, 0, 1, 1, 0, 0, 0]

In [5]:
#Inputs how many rows to be processed
inputs = []
lengthInput = 4000
print("Total length of inputs: " + str(lengthInput))

for r in range(lengthInput):
    inputs.append(tempinputs[r])
    
inputs = [[x[0:8], x[8:]] for x in inputs]

Total length of inputs: 4000


In [6]:
net = Network(8, 60, 1)

#desired output/target
def des(inputs):
    re = int(inputs[9])
    return [re]

In [7]:
net.set_inputs(inputs)

In [8]:
net.reset()
net.train(max_training_epochs=1500, report_rate=250, tolerance = .25)

--------------------------------------------------
Training for max trails: 1500 ...
Epoch: 0 TSS error: 2190.14764442 %correct: 0.21525
Epoch: 250 TSS error: 341.57984451 %correct: 0.65225
Epoch: 500 TSS error: 340.024675507 %correct: 0.65525
Epoch: 750 TSS error: 340.604173309 %correct: 0.65475
Epoch: 1000 TSS error: 339.872056034 %correct: 0.64775
Epoch: 1250 TSS error: 339.682940344 %correct: 0.65275
Epoch: 1500 TSS error: 339.523031001 %correct: 0.65925
--------------------------------------------------
Epoch: 1500 TSS error: 339.523031001 %correct: 0.65925


In [9]:
net.test()

--------------------------------------------------
Test:
******************************
Input : [1, 1, 0, 0, 1, 1, 0, 0]
Output: [ 0.25467061] 
Target: [0] Incorrect
******************************
Input : [1, 0, 1, 0, 1, 1, 0, 0]
Output: [ 0.26970914] 
Target: [1] Incorrect
******************************
Input : [1, 0, 1, 0, 1, 0, 1, 0]
Output: [ 0.76901609] 
Target: [0.5] Incorrect
******************************
Input : [0, 0, 1, 0, 0, 1, 0, 0]
Output: [ 0.24621069] 
Target: [0] Correct
******************************
Input : [1, 1, 0, 0, 0, 1, 0, 0]
Output: [ 0.07737345] 
Target: [0] Correct
******************************
Input : [1, 0, 1, 0, 1, 1, 0, 0]
Output: [ 0.26970914] 
Target: [0.5] Correct
******************************
Input : [1, 0, 1, 0, 1, 1, 0, 0]
Output: [ 0.26970914] 
Target: [0.5] Correct
******************************
Input : [1, 0, 1, 0, 1, 1, 0, 0]
Output: [ 0.26970914] 
Target: [0] Incorrect
******************************
Input : [1, 1, 0, 0, 0, 1, 0, 0]
Output: [

In [42]:
len(tempinputs) - 4000

2149

In [24]:
wresults = []
bresults = []
wtest = []
btest = []

for p in range(len(tempinputs) - 4000):
    val = tempinputs[p+4000]
    print(val)
    #White
    if val[4] == 0:
        wtest.append(val[0:8])
    #black
    if val[4] == 1:
        btest.append(val[0:8])
    
for l in wtest:
    wresults.append(net.propagate(l))
    
for m in btest:
    bresults.append(net.propagate(m))

wFalseCount = 0
bFalseCount = 0
wMidCount = 0
bMidCount = 0
wTrueCount = 0
bTrueCount = 0
lowCountTrainData = 0
medCountTrainData = 0
highCountTrainData = 0

for wr in wresults:
    if wr < .25:
        wFalseCount += 1

for br in bresults:
    if br < .25:
        bFalseCount += 1
        
for wr in wresults:
    if .25 <= wr <= .75:
        wMidCount += 1

for br in bresults:
    if .25 <= br <= .75:
        bMidCount += 1
        
for wr in wresults:
    if wr > .75:
        wTrueCount += 1

for br in bresults:
    if br > .75:
        bTrueCount += 1
        
for rowz in tempinputs:
    if rowz[8:] == [0]:
        lowCountTrainData += 1
        
for rowz in tempinputs:
    if rowz[8:] == [.5]:
        medCountTrainData += 1
        
for rowz in tempinputs:
    if rowz[8:] == [1]:
        highCountTrainData += 1

print("\n")  
print("wFalseCount")   
print(wFalseCount)

print("\n")  
print("bFalseCount")   
print(bFalseCount)
        
print("falseCountTrainData")   
print(falseCountTrainData)

print("midCountTrainData")   
print(midCountTrainData)

print("trueCountTrainData")   
print(lowCountTrainData)

print("trueCountTrainData")   
print(medCountTrainData)

print("trueCountTrainData")   
print(highCountTrainData)

[1, 0, 1, 0, 1, 0, 1, 0, 1]
[1, 0, 0, 1, 1, 0, 1, 0, 0]
[1, 1, 0, 0, 0, 1, 0, 0, 0]
[1, 1, 0, 0, 1, 1, 0, 0, 0]
[1, 0, 1, 0, 0, 1, 0, 0, 0.5]
[1, 0, 0, 1, 1, 0, 1, 0, 0.5]
[1, 1, 0, 0, 1, 0, 0, 1, 0.5]
[1, 0, 1, 0, 1, 1, 0, 0, 0]
[1, 1, 0, 0, 0, 1, 0, 0, 0]
[0, 0, 1, 0, 1, 1, 0, 0, 0.5]
[0, 0, 1, 0, 1, 1, 0, 0, 0.5]
[1, 0, 0, 1, 0, 0, 1, 0, 0.5]
[1, 0, 1, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 1, 1, 0, 1, 0, 1]
[1, 0, 1, 0, 0, 1, 0, 0, 0]
[1, 0, 1, 0, 1, 0, 1, 0, 0.5]
[1, 0, 1, 0, 1, 1, 0, 0, 1]
[1, 0, 1, 0, 1, 0, 0, 1, 1]
[0, 0, 1, 0, 0, 1, 0, 0, 0]
[1, 0, 1, 0, 1, 1, 0, 0, 1]
[1, 0, 1, 0, 1, 1, 0, 0, 0]
[1, 0, 0, 1, 0, 0, 1, 0, 0]
[1, 0, 1, 0, 1, 0, 1, 0, 1]
[1, 1, 0, 0, 1, 1, 0, 0, 0]
[1, 0, 1, 0, 0, 1, 0, 0, 0.5]
[1, 0, 0, 1, 1, 0, 1, 0, 0.5]
[1, 0, 1, 0, 1, 1, 0, 0, 0.5]
[1, 0, 0, 1, 1, 0, 1, 0, 0.5]
[1, 0, 1, 0, 1, 0, 0, 1, 1]
[1, 1, 0, 0, 1, 1, 0, 0, 0]
[1, 0, 1, 0, 0, 1, 0, 0, 0]
[1, 0, 1, 0, 0, 0, 1, 0, 1]
[1, 0, 1, 0, 1, 0, 0, 1, 1]
[1, 0, 1, 0, 1, 1, 0, 0, 0.5]
[0, 0, 1, 0, 1, 1, 0, 0,

In [25]:
def getBias(falseCount, testSet, countTrue):
    if len(testSet) == 0:
        return 0
    numer = falseCount/len(testSet)
    print(numer)
    demo = countTrue/lengthInput
    print(demo)
    print("Bias ratio given test dataset: ")
    return numer/demo

In [26]:
print("White cases: ")
print(getBias(wFalseCount, wtest, lowCountTrainData))

White cases: 
0.7523584905660378
0.78025
Bias ratio given test dataset: 
0.9642531119077703


In [27]:
print("Black Cases: ")
print(getBias(bFalseCount, btest, lowCountTrainData))

Black Cases: 
0.14757878554957723
0.78025
Bias ratio given test dataset: 
0.18914294847751006


In [28]:
print("White cases: ")
print(getBias(wMidCount, wtest, medCountTrainData))

White cases: 
0.19693396226415094
0.43175
Bias ratio given test dataset: 
0.45612961728813184


In [29]:
print("black cases: ")
print(getBias(bMidCount, btest, medCountTrainData))

black cases: 
0.541890853189854
0.43175
Bias ratio given test dataset: 
1.2551033079093317


In [30]:
print("White cases: ")
print(getBias(wTrueCount, wtest, highCountTrainData))

White cases: 
0.05070754716981132
0.32525
Bias ratio given test dataset: 
0.1559032964483054


In [31]:
print("black cases: ")
print(getBias(bTrueCount, btest, highCountTrainData))

black cases: 
0.31053036126056877
0.32525
Bias ratio given test dataset: 
0.9547436164813798


In [35]:
wresults = []
bresults = []
wtest = []
btest = []

for p in range(len(tempinputs) - 4000):
    val = tempinputs[p+4000]
    #White
    val[4] = 0
    wtest.append(val[0:8])
    #black
    val[4] = 1
    btest.append(val[0:8])
    
for l in wtest:
    wresults.append(net.propagate(l))
    
for m in btest:
    bresults.append(net.propagate(m))

wFalseCount = 0
bFalseCount = 0
wMidCount = 0
bMidCount = 0
wTrueCount = 0
bTrueCount = 0
lowCountTrainData = 0
medCountTrainData = 0
highCountTrainData = 0

for wr in wresults:
    if wr < .25:
        wFalseCount += 1

for br in bresults:
    if br < .25:
        bFalseCount += 1
        
for wr in wresults:
    if .25 <= wr <= .75:
        wMidCount += 1

for br in bresults:
    if .25 <= br <= .75:
        bMidCount += 1
        
for wr in wresults:
    if wr > .75:
        wTrueCount += 1

for br in bresults:
    if br > .75:
        bTrueCount += 1
        
for rowz in tempinputs:
    if rowz[8:] == [0]:
        lowCountTrainData += 1
        
for rowz in tempinputs:
    if rowz[8:] == [.5]:
        medCountTrainData += 1
        
for rowz in tempinputs:
    if rowz[8:] == [1]:
        highCountTrainData += 1

print("\n")  
print("wFalseCount")   
print(wFalseCount)

print("\n")  
print("bFalseCount")   
print(bFalseCount)

print("lowCountTrainData")   
print(lowCountTrainData)

print("medCountTrainData")   
print(medCountTrainData)

print("highCountTrainData")   
print(highCountTrainData)



wFalseCount
1329


bFalseCount
376
lowCountTrainData
3121
medCountTrainData
1727
highCountTrainData
1301


In [36]:
print("White cases: ")
print(getBias(wFalseCount, wtest, lowCountTrainData))

White cases: 
0.6184271754304328
0.78025
Bias ratio given test dataset: 
0.7926013142331725


In [37]:
print("Black Cases: ")
print(getBias(bFalseCount, btest, lowCountTrainData))

Black Cases: 
0.17496510004653326
0.78025
Bias ratio given test dataset: 
0.2242423582781586


In [38]:
print("White cases: ")
print(getBias(wMidCount, wtest, medCountTrainData))

White cases: 
0.281991624011168
0.43175
Bias ratio given test dataset: 
0.6531363613460752


In [39]:
print("black cases: ")
print(getBias(bMidCount, btest, medCountTrainData))

black cases: 
0.5835272219637041
0.43175
Bias ratio given test dataset: 
1.3515395992210864


In [40]:
print("White cases: ")
print(getBias(wTrueCount, wtest, highCountTrainData))

White cases: 
0.09958120055839925
0.32525
Bias ratio given test dataset: 
0.3061681800411968


In [41]:
print("black cases: ")
print(getBias(bTrueCount, btest, highCountTrainData))

black cases: 
0.2415076779897627
0.32525
Bias ratio given test dataset: 
0.7425293712214073


In [38]:
len(bresults)

2149